In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import scipy.stats as stats
from toolz.curried import *
import itertools

import gc_model as gc
import hic_analysis as hic
import visualize

In [3]:
@curry
@excepts(FileNotFoundError)
def fit_dir(d, n):
    return np.load(f'{d}/{n}.npz')

chr19_eye = fit_dir('rao2014/chr19-eye')
chr20_eye = fit_dir('rao2014/chr20-eye')

In [4]:
@curry
def unpack_args(f, args_list):
    return f(*args_list)

unzip = lambda zipped: zip(*zipped)

In [5]:
input_data_chr19 = np.load('rao2014/Rao2014-chr19.npy')
input_data_chr20 = np.load('rao2014/Rao2014-chr20.npy')

In [191]:
%matplotlib qt5
visualize.all_figures(input_data_chr20, chr20_eye(2).values(), chr20_eye(3).values(), histones_dir='histone_modifications/')

/Users/hkariti/repo/technion/Compartments/hic_analysis.py:94: RuntimeWarning: Mean of empty slice
  diag_mean = np.nanmean(diag)


In [28]:
input_subset = input_data_chr20[840:840+300, :][:, 840:840+300]

In [29]:
fit_subset = gc.fit(input_subset, number_of_states=2, weights_shape='eye')

In [108]:
def plot_merged(data, fit):
    generated = gc.generate_interactions_matrix(*fit)
    merged = hic.merge_by_diagonal(hic.normalize_distance(hic.preprocess(data)), hic.normalize_distance(generated))
    plt.imshow(hic.safe_log(merged))
    
generate_normalized = compose(hic.safe_log, hic.normalize_distance, unpack_args(gc.generate_interactions_matrix))

In [12]:
%matplotlib qt5

In [217]:
input_subsets = compose(list, map(lambda d: input_data_chr20[840-d:840+400, 840-d:840+400]))(range(100, 1000, 100))
fit_eye2 = partial(gc.fit, number_of_states=2, weights_shape='eye')
subset_fits = compose(list, map(fit_eye2))(input_subsets)

In [219]:
for input_subset, f in zip(input_subsets, subset_fits):
    d = input_subset.shape[0]
    visualize.hic_figure(input_subset,
                         f[0],
                         f[1], f[2])
    plt.title(f'840+{d}')

In [210]:
d=200
input_subset = input_data_chr20[840:840+d, :][:, 840:840+d]
fit_subset = gc.fit(input_subset, number_of_states=2, weights_shape='eye')

In [211]:
visualize.plot_data(plt.figure(3).axes[0], input_subsets[2], subset_fits[2][0], subset_fits[2][2], subset_fits[2][1])
#plt.figure(3).axes[0].imshow(generate_normalized(subset_fits[2])[:200, :200], vmin=vmin, vmax=vmax)
plt.figure(3).axes[0].set_xlim(0, 300)
plt.figure(3).axes[0].set_ylim(300, 0)
visualize.plot_probabilities(plt.figure(3), plt.figure(3).axes[1], subset_fits[2][0])

IndexError: list index out of range

In [166]:
ax = plt.figure(3).axes[0].images[-1]

In [223]:
fit_2states_highacc = gc.fit(input_data_chr20, number_of_states=2, weights_shape='eye')
fit_3states_highacc = gc.fit(input_data_chr20, number_of_states=3, weights_shape='eye')

In [224]:
input_data_chr20.shape

(1261, 1261)

In [226]:
list(chr20_eye(2).keys())

['lambdas', 'weights', 'alpha']